# Лабораторна робота 2

## Теоритичні матеріали

Математична статистика, це область науки яка займається, тим, що за допомогою математичних методі намагєтсья вивести правила або зробить висновки, щодо наявних статистичних даних.

Задачами математичної статистики є:

* Формування та аналіз основних числових характеристик випадкових вибірок
* Оцінка розподілу статистичних імовірностей та його параметрів, 
* Статистична перевірка гіпотез
* Вивчення статистичної залежності
* Формулювання статистичних висновків та звітів тощо

Нижче наведено список основних характерстик, що використовуються в матиматичній статистиці:

* Узагальні(середні) характеристики вибірки:
    * Матиматичне сподівання - середне значення у вибірці. $$ E[X] = \int\limits_U X(w)P(dw)  $$
    * Медіана - значення, які зустрічається в вибірці найбільш часто. $$ x - медіана, якщо \, F(x) = 1/2 $$
* Характеристи розподілу:
    * Дисперсія - характеристика, що показує міру розсіювання вимірювань відносно матиматичного сподівання $$ D[X] = \sqrt {\frac{1}{N}\sum\limits_{i = 1}^N {\left( {x_i - \bar x} \right)^2 } } $$
    * Стандартне відхилення - характериситка, що міру стандартного відхилення від матиматичного сподівання, що робить її популярною для побудови довірчного інтервалу для вибірки $$ std[X] = \sqrt{D[X]} $$
    * Розмах - характеристика, що дозволяє вказати границі множини значень, визначається як різниця максимального і мінімального елемента. $$ R = x_{max} - x_{min} $$
    * Квантиль, це значення для яких функція розпоіду становить 25% (перший), 50% (другий або медіанна), 75%(третій). $$ F_X(x) := Pr(X \leq x) = p $$

## Ініцілізація

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels import robust
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import OLSInfluence
import seaborn as sns

table = pd.read_csv("/kaggle/input/test-file/tested.csv")

table.dropna(subset = ["Fare"], inplace = True)

table.head()

## Хід роботи

### Завдання 1: Розрахувати показники оцінки центральних значень варіаційного ряду з урахуванням викидів та робастних оцінок

На цьому кроці ми маємо дізнатись основні статистичні харектеристики вибірки

In [ ]:
def weighted_median(df, val, weight):
    df_sorted = df.sort_values(val)
    cumsum = df_sorted[weight].cumsum()
    cutoff = df_sorted[weight].sum() / 2.
    return df_sorted[cumsum >= cutoff][val].iloc[0]

FareGroup = table[["Fare", "Pclass"]].groupby("Pclass").agg(FareCount = ("Fare", "count"), FareMean = ("Fare", "mean"))

values = table["Fare"].dropna()

pd.DataFrame.from_dict(
    {
    "Середнє значення" : values.mean(),
    "Середнє зважене" : np.average(FareGroup["FareMean"], weights = FareGroup["FareCount"]),
    "Середня усічена" : stats.trim_mean(values, 0.1),
    "Медіана" : values.median(),
    "Медіана зважена": weighted_median(FareGroup, "FareMean", "FareCount"),
    "Робастне значення" : robust.scale.mad(values)
    }, orient = "index", columns = ["Значення"])

З цих розрахунків ми можемо зробити наступний висновок:

* В вибірці найбільш частіше зустрічається значення 14.54 (Медіана)
* Середнє значення для цієї вибірки - 35.63
* Для усіченої вибірки(відкинуто екстремальні значення) середнє - 22
* Робестне значення, що показує робастність(стійкаість вибірки до викидів та тощо) становить - 10.12 

In [ ]:
f = 'Fare ~ PassengerId'
model = ols(formula=f, data=table).fit()

cook_distance = OLSInfluence(model).cooks_distance
(distance, p_value) = cook_distance

sns.scatterplot(x = table.PassengerId, y = table.Fare, hue=distance, size=distance, sizes=(50, 200), edgecolor='black', linewidth=1)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlabel('PassengerId', fontsize=14)
plt.ylabel('Fare', fontsize=14)
plt.title('Cook\'s distance', fontsize=20);

Для оцінки викидів було застосовано візуалізацію дистанції кука, ця величина показує вплив кожного елемента на вибірку. В ній адекватні мають бути розташовані щільно, що дозволяє легко побачити викиди на цьому графіку 

### Завдання 2. Оцінити показники варіабельності варіаційного ряду

На цьому кроці ми маємо розрахувати параметри, які допоможу зробить висоновк, про варіативність вибірки

In [ ]:
pd.DataFrame.from_dict(
    {
        "Стандартне відхилення" : values.std(),
        "Дисперсія" : values.var(),
        "1 квантиль" : values.quantile(0.25),
        "3 квантиль" : values.quantile(0.75),
        "Межквартільний розмах" : values.quantile(0.75) - values.quantile(0.25),
        "Найменший елемент" : values.min(),
        "Найбільший елемент" : values.max(),
        "Розмах вибірки" : values.max() - values.min(),
        "Середнє абсолютне відхилення" : values.mad() 
    }, orient = "index", columns = ["Значення"])

З цього аналізу ми можемо зробити декілька висновків:

* На основі розмаху вибірки(512) ми можемо говорити, що множина можливих значень є достанньо великою
* Хоча множина можливих значень є великою, але саме розташування елементів є достанньо щільним. Особливо гарно, це характеризує 3 квантиль, який говорить 75% всіх елементів цієї вибірки не перебільшують 31.5, а також значення стандартного відхилоення(56), яке показує рівень розсіювання значень відносно середнього(в нашому випадку 35.6)

In [ ]:
bins = pd.cut(values,
       bins = 6,
       labels=['Very low', "Low",
               'Mid', "Upper",
               'High', "Luxery"]).value_counts().plot(kind = "bar", xlabel = "Ціна білета", ylabel = "Продано білетів")

На цій діаграмі можна побачить, що дійсно більшість елементів розташовані в одному діапазону, що становить лише 1\6 всієї вибірки, що підтверджує висновки, що були отримані раніше

### Завдання №3:  Побудувати базові статистичні графіки

На цьому кроці нашою задечою є побудова базових графіків розподілу: гістограма, графік(plot) та ящик з вусикам. Ці графіки є достанньо простими, але дозволяють наглядно показати особливості розподілу

In [ ]:
fig, axes = plt.subplots(1, 3, figsize = (10, 6))

fig.tight_layout()

fig = values.plot.hist(ax = axes[0], title = "Histogram")
fig = values.plot.density(ax = axes[1], title = "Density")
fig = values.plot.box(ax = axes[2], title = "Box")

plt.subplots_adjust(left=-1)

plt.show()

Кожен з ции графіків відображає розподіл ціни за квиток, на них видно, що більшість, що були продані коштували до 100, після цього можна побачити набагто меншу групу в діапазоні 200-300. Окрім цього :
* За 1 і 2 графіком можна побачити, що цей розподіл схожий на показниковий закон розподілу
* 3 графік може допомогти в оцінці викидів

### Завдання 4. Оцінити елементарні метрики парної залежності змінних

На цьому кроці нашою задачею є дослідить рівень кореляції(залежність одного параметру від іншого) в нашій вибірці. Для цього ми зробимо таблицю з коефіціентом залежності, а також візуаліазацію для допомого HeatMap для більшої наглядності

In [ ]:
cor = table.corr()

sns.heatmap(cor)

cor

З результатів аналізу можна побачити, що параметри цієї вибірки не мають значимого зв'язку між собою. Хоча варто відмітить, що існує помірна корелція між Pclass(Клас пасажира) та Fare(ціна за квиток)

### Завдання 5. Проаналізувати статистичні характеристики категорійних змінних

На цьому кроці нашою задачею є дослідити, на які категорії можна поділити дані вибірки(в нашому випадку пасажирів) та дати оцінку особливостях розподілу в цих категоріях. Для початку ми можемо виділити категоральні параметри шляхом підрахунку унікальних значеннь кожного параметра

In [ ]:
pd.DataFrame(table.nunique())

Зрозуімло, що серед всіх параметрів категоріальними є: Sex, Pclass, Embarked, Survived

In [ ]:
pd.crosstab([table["Sex"], table["Pclass"], table["Embarked"]], table["Survived"])

За цією таблицею спряженості можна зробить видно, що:
* Всі жінки спаслись, а чоловіки потонули
* Більшість пасажирів належало до 3 класа, а також зайшли на борт в порту S

## Висновки

В рамках цієї роботи за допомогою інструменту Jupyter та методів з матиматичної статистики ми дослідили наявну вибірку. Цей процес включав такі етапи:

1. Оцінка середніх характерстик вибірки, що дало нам можливість зробить узагальнений висновок по цій вибірці
2. Оцінка характерстик варібельності, що дало змогу оцінити певні особливості розподілу в цій вибірці
3. Побудова стандартних графіків, що дозволило наочно продемонструвати особлиовсті розподілу
4. Дослідження кореляції, яке дозволило проаналізувати залежності між параметрами цієї вибірки
5. Аналіз категорільних зміних, в рамках якого було виокремлено категоріальні змінні, та побудовано спряжену таблицю, яка дозвоила порівняти кільсть людей, яким вдалось спастить по всіх групах